Imports


In [174]:
import cv2
import numpy as np
import pandas as pd
import os
import math

In [175]:
#Path to the raw images
path_sameed_raw = "C:\\Users\\Sameed\\Desktop\\ML_Overall_Work\\Dataset\\Sameed" 
path_unknown_raw = "C:\\Users\\Sameed\\Desktop\\ML_Overall_Work\\Dataset\\Unknown"

#Get the names of all the images
images_sameed = os.listdir(path_sameed_raw) 
images_unknown = os.listdir(path_unknown_raw)

#Store the paths of all the cropped images we will save later
paths_sameed_cropped = []
paths_unknown_cropped = []


In [176]:
sameed_ages = []

for img in enumerate(images_sameed):

    # To ignore the first 2 folders in the directory
    if(img[1] == "Cropped" or img[1] == "Detected"):
        continue

    age = ""
    for i in range(0,len(img[1])):
        if(img[1][i] == "_"):
            break
        else:
            age += img[1][i]

    sameed_ages.append(age)

    #Open the image and extract the coordinates of the detected face
    image = cv2.imread(path_sameed_raw + "\\" + img[1])
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
    faces = faceCascade.detectMultiScale(
        gray,
        scaleFactor=1.3,
        minNeighbors=3,
        minSize=(64, 64)
    )
    
    # This is to select the highest detected area, in case there are false positives.
    # As in this scenario it is highly likely that the greater area will contain the face rather then
    # the smaller falsepostive

    if(len(faces) > 1):
        cur_max = 0
        face = 0

        for (x, y, w, h) in faces:
            length = math.sqrt((x - (x+w))**2 + (y - (y+h))**2 )
            
            if length > cur_max:
                cur_max = length
                face = [x,y,w,h]

        faces = np.array([face])

    for (x, y, w, h) in faces:
    
        cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2) # Draw a rectangle around the detected area (face)

        roi_color = image[y+2:y+2 + h-3, x+2:x+2 + w-3] # The pixel area containing the face
        
        # Path where image will be saved
        cropped_img_path = path_sameed_raw + "\\Cropped" + "\\Sameed_" + str(img[0]+1) + "_.jpg"
        paths_sameed_cropped.append(cropped_img_path.replace("\\",chr(92)))
        

        # Resize each image to 64x64 before saving it to the path
        cropped_img = cv2.resize(roi_color, (64,64))
        cv2.imwrite(cropped_img_path, cropped_img)

    # Save the uncropped image with detected face
    detected_img_path = path_sameed_raw + "\\Detected" + "\\Sameed_" + str(img[0]+1) + "_.jpg"
    cv2.imwrite(detected_img_path, image)


In [177]:
unknown_ages = []

#Store all the paths for the unknown images
for unknowns in enumerate(images_unknown):
    if(unknowns[1] == "Cropped"):
        continue
    
    age = ""
    for i in range(0,len(unknowns[1])):
        if(unknowns[1][i] == "_"):
            break
        else:
            age += unknowns[1][i]

    unknown_ages.append(age)

    path = path_unknown_raw + "\\" + unknowns[1]
    image = cv2.imread(path)

    img_resize = cv2.resize(image, (64,64))
    
    new_path = path_unknown_raw+"\\"+"Cropped"+"\\Unknown_"+str(unknowns[0]+1)+"_.jpg"
    cv2.imwrite(new_path, img_resize)

    np = new_path.replace("\\",chr(92))
    paths_unknown_cropped.append(np)

In [178]:
%store paths_unknown_cropped
%store paths_sameed_cropped
%store unknown_ages
%store sameed_ages

Stored 'paths_unknown_cropped' (list)
Stored 'paths_sameed_cropped' (list)
Stored 'unknown_ages' (list)
Stored 'sameed_ages' (list)
